In [1]:
import pandas as pd

from config import DATA_HOURLY
from config import DATA_YEARLY

In [3]:
df_hourly = pd.read_excel(DATA_HOURLY)
df_yearly = pd.read_excel(DATA_YEARLY)

In [42]:
df_hourly.ffill()

,Hour,PROEEWINDON_ENTSOE,PROEESOL_ENTSOE,year,wind_profile,solar_profile,Year
0,2020-01-01 00:00:00.000,138.0,0.0,2020,0.563870,0.000000,2020
1,2020-01-01 01:00:00.000,142.0,0.0,2020,0.580214,0.000000,2020
2,2020-01-01 02:00:00.000,153.0,0.0,2020,0.625160,0.000000,2020
3,2020-01-01 03:00:00.000,134.0,0.0,2020,0.547526,0.000000,2020
4,2020-01-01 04:00:00.000,110.0,0.0,2020,0.449461,0.000000,2020
...,...,...,...,...,...,...,...
43825,2024-12-31 00:59:59.991,188.0,0.0,2024,0.534880,0.000000,2024
43826,2024-12-31 01:59:59.991,170.0,0.0,2024,0.483668,0.000000,2024
43827,2024-12-31 02:59:59.991,124.0,0.0,2024,0.352793,0.000000,2024
43828,2024-12-31 03:59:59.991,67.0,1.0,2024,0.190622,0.000826,2024


In [6]:
df_yearly.head(

,Year,"WIND Produced, MWh","SOLAR Produced, MWh","WIND - Installed, MW","SOLAR - Installed, MW","WIND - Annual prod MWh, MW","SOLAR - Annual prod MWh, MW",Capture Factor
0,2020,881025.5,127716.0,316.66,170,3600,751.270588,NaN
1,2021,776160.5,329449.0,337.66,395,3063,834.048101,NaN
2,2022,684600.5,559105.0,376.66,520,2420,1075.201923,NaN
3,2023,804053.0,664047.0,381.36,812,2800,817.791872,NaN
4,2024,1160067.5,992366.5,456.36,1210,3300,820.137603,NaN


In [14]:
df_hourly['Hour'] = pd.to_datetime(df_hourly['Hour'])
df_hourly['Year'] = df_hourly['Hour'].dt.year

# Initialize new columns
df_hourly['wind_profile'] = 0.0
df_hourly['solar_profile'] = 0.0

years = df_yearly['Year'].unique()

In [25]:
df_hourly.head()

,Hour,PROEEWINDON_ENTSOE,PROEESOL_ENTSOE,year,wind_profile,solar_profile,Year
0,2020-01-01 00:00:00,138.0,0.0,2020,0.563870,0.0,2020
1,2020-01-01 01:00:00,142.0,0.0,2020,0.580214,0.0,2020
2,2020-01-01 02:00:00,153.0,0.0,2020,0.625160,0.0,2020
3,2020-01-01 03:00:00,134.0,0.0,2020,0.547526,0.0,2020
4,2020-01-01 04:00:00,110.0,0.0,2020,0.449461,0.0,2020


In [36]:
for year in years:
    mask = df_hourly['Year'] == year
    wind_hourly = df_hourly.loc[mask, 'PROEEWINDON_ENTSOE']
    solar_hourly = df_hourly.loc[mask, 'PROEESOL_ENTSOE']

    wind_yearly = df_yearly.loc[df_yearly['Year'] == year, 'WIND - Annual prod MWh, MW'].values[0]
    solar_yearly = df_yearly.loc[df_yearly['Year'] == year, 'SOLAR - Annual prod MWh, MW'].values[0]

    df_hourly.loc[mask, 'wind_profile'] = wind_hourly / (wind_hourly.sum() / wind_yearly)
    df_hourly.loc[mask, 'solar_profile'] = solar_hourly / (solar_hourly.sum() / solar_yearly)

df_profiles = df_hourly[['Hour', 'Year', 'wind_profile', 'solar_profile']]

In [44]:
df_profiles.tail()

,Hour,Year,wind_profile,solar_profile
43825,2024-12-31 00:59:59.991,2024,0.534880,0.000000
43826,2024-12-31 01:59:59.991,2024,0.483668,0.000000
43827,2024-12-31 02:59:59.991,2024,0.352793,0.000000
43828,2024-12-31 03:59:59.991,2024,0.190622,0.000826
43829,2024-12-31 04:59:59.991,2024,0.165016,0.000000
